# KEGG BRITE functional hierarchies
In this notebook, a mapping procedure is carried out with the purpose of creating a final table that links the hierarchical functional information contained in KEGG BRITE database to the genes present in the PanCancer gene-expression dataset.

In [1]:
import pandas as pd
import numpy as np

##  KEGG and PanCancer data loading

- Data coming from KEGG database is fetched on `KEGG_BRITE_Data_Obtaining` notebook.
- Data coming from PanCancer dataset is obtained from a previous project (see [A Transfer-Learning Approach to Feature Extraction from Cancer Transcriptomes with Deep Autoencoders](https://link.springer.com/chapter/10.1007%2F978-3-030-20521-8_74)). In this previous work, the ~9K most variably expressed genes were retained. However, in this case, the 20K most variably expressed genes have been kept (also using standard deviation (SD) and median absolute deviation (MAD) metrics).

In [2]:
%%time
# KEGG information
hsa = pd.read_csv('/mnt/ext/almacen/comun/mcabello/TFG/DL_CancerSurvivalPrediction/data/KEGG/KEGG_gene_hsa.csv', sep='\t', 
                  engine='python', header=None, index_col=None, names=['keggId', 'geneName'])
hsa_brite = pd.read_csv('/mnt/ext/almacen/comun/mcabello/TFG/DL_CancerSurvivalPrediction/data/KEGG/KEGG_hsa_brite.csv', 
                        sep='\t', engine='python', header=None, index_col=None, names=['keggBriteId', 'keggId'])
brite = pd.read_csv('/mnt/ext/almacen/comun/mcabello/TFG/DL_CancerSurvivalPrediction/data/KEGG/KEGG_brite.csv', 
                    engine='python', header=0, index_col=0)

# PanCancer gene-expression dataset
brca_ex = pd.read_hdf("/mnt/ext/almacen/comun/Francis/almacen/DeepLearning-Bioinformatics/data/PanCancer/mad_filter_pancan_all_TCGA_20.h5", 
                      key = "brca")
non_brca_ex = pd.read_hdf("/mnt/ext/almacen/comun/Francis/almacen/DeepLearning-Bioinformatics/data/PanCancer/mad_filter_pancan_all_TCGA_20.h5", 
                          key = "non_brca")

CPU times: user 202 ms, sys: 457 ms, total: 659 ms
Wall time: 658 ms


As a sanity check procedure, we check if the samples contained in the clinical outcomes datasets are also contained in the gene expression datasets:

In [3]:
Y_pancan_non_brca_surv = pd.read_hdf("../data/PanCancer/non_BRCA_pancan.h5", key="sample_clinical")
Y_pancan_non_brca_surv.shape

(9285, 33)

In [4]:
len(Y_pancan_non_brca_surv.index.intersection(list(map(lambda x: x.split('_')[0], non_brca_ex.index))))

9285

In [5]:
Y_pancan_brca_surv = pd.read_hdf("../data/PanCancer/BRCA_pancan.h5", key="survival_outcome")
Y_pancan_brca_surv.shape

(1211, 33)

In [6]:
len(Y_pancan_brca_surv.index.intersection(list(map(lambda x: x.split('_')[0], brca_ex.index))))

1211

Get ENSEMBL IDs:

In [7]:
ens_genes = brca_ex.columns

In [8]:
print("Initial number of genes: {}".format(len(ens_genes)))

Initial number of genes: 20000


In [9]:
print("Total number of samples (patients): {}".format(brca_ex.shape[0] + non_brca_ex.shape[0]))

Total number of samples (patients): 10535


## MAPPING: From Ensembl Gene ID to KEGG BRITE functional hierarchy
KEGG relates Hugo-gene-names to an internal id (KEGG gene id) that is useful to find out the corresponding KEGG BRITE ids (hierarchical functional annotations) of each gene. Because of genes in PanCancer dataset are indexed by ENSEMBL-ids, a mapping procedure needs to be performed, that associates ENSEMBL-id to Hugo-gene-name, Hugo-gene-name to KEGG gene id and KEGG gene id to KEGG BRITE id.

### Mapping: Ensembl Gene ID to  Hugo Gene Name

We load a table relating ENSEMBL-ids and Hugo-gene-names, downloaded from: https://raw.githubusercontent.com/jvivian/docker_tools/master/gencode_hugo_mapping/attrs.tsv

In [10]:
%%time
hugo_ens = pd.read_csv('/mnt/ext/almacen/comun/mcabello/TFG/DL_CancerSurvivalPrediction/data/hugo_ensembl.tsv', 
                       sep='\t', engine='python', index_col=None)

CPU times: user 836 ms, sys: 92 ms, total: 928 ms
Wall time: 950 ms


In [11]:
hugo_ens = hugo_ens[~hugo_ens.duplicated(subset=['geneId', 'geneName'])]
hugo_ens.shape

(65670, 13)

We only select genes contained in the expression datasets:

In [12]:
hugo_ens = hugo_ens[hugo_ens['geneId'].isin(ens_genes)]
hugo_ens.shape

(20000, 13)

In [13]:
gene_mapping = hugo_ens[['geneId', 'geneName']]

In [14]:
gene_mapping.head()

,geneId,geneName
2,ENSG00000227232.5,WASH7P
12,ENSG00000238009.6,RP11-34P13.7
13,ENSG00000239945.1,RP11-34P13.8
18,ENSG00000233750.3,CICP27
19,ENSG00000268903.1,RP11-34P13.15


In [15]:
# Sanity check
sum(gene_mapping.duplicated())

0

In [16]:
print("Number of Hugo genes contained in the expression datasets:", len(set(gene_mapping['geneName'])))

Number of Hugo genes contained in the expression datasets: 19959


In [18]:
print("Number of ENSEMBLE genes contained in the expression datasets:", len(set(gene_mapping['geneId'])))

Number of ENSEMBLE genes contained in the expression datasets: 20000


### Mapping: Hugo Gene Name to KEGG gene id

In [19]:
# Check if we can split using the next two characters
sum(map(lambda x: ',' in x or ';' in x, gene_mapping.geneName))

0

In [20]:
%%time
import itertools

# We create a dataframe from a list with all (keggId, hugoGeneName) pairs contained in hsa dataframe
hsa_hugo = pd.DataFrame(data=list(itertools.chain.from_iterable(hsa.apply(
    lambda x: list(zip(itertools.repeat(x[0]), x[1].replace(';', ',').split(', '))), axis=1))), 
                        columns=['keggId', 'geneName'])

CPU times: user 758 ms, sys: 0 ns, total: 758 ms
Wall time: 757 ms


In [21]:
print(hsa_hugo.shape)
hsa_hugo.head()

(126869, 2)


,keggId,geneName
0,hsa:4549,RNR1
1,hsa:4549,MTRNR1
2,hsa:4549,MT-RNR1
3,hsa:4549,s-rRNA
4,hsa:4550,RNR2


In [22]:
# Sanity check
sum(hsa_hugo.duplicated())

86

In [23]:
# Remove duplicated rows
hsa_hugo = hsa_hugo[~hsa_hugo.duplicated()]

In [24]:
gene_mapping = pd.merge(gene_mapping, hsa_hugo, on='geneName')

In [25]:
gene_mapping.head()

,geneId,geneName,keggId
0,ENSG00000227232.5,WASH7P,hsa:653635
1,ENSG00000226210.3,WASH7P,hsa:653635
2,ENSG00000177757.2,FAM87B,hsa:400728
3,ENSG00000225880.5,LINC00115,hsa:79854
4,ENSG00000230368.2,FAM41C,hsa:284593


In [26]:
# Sanity check
sum(gene_mapping.duplicated())

0

In [27]:
print("Number of Hugo genes contained in the expression datasets that are associated with KEGG IDs:", 
      len(set(gene_mapping['geneName'])))

Number of Hugo genes contained in the expression datasets that are associated with KEGG IDs: 12475


In [28]:
print("Number of ENSEMBLE genes contained in the expression datasets that are associated with KEGG IDs:", 
      len(set(gene_mapping['geneId'])))

Number of ENSEMBLE genes contained in the expression datasets that are associated with KEGG IDs: 12506


### Mapping: KEGG gene id to KEGG BRITE id

In [29]:
gene_mapping = pd.merge(gene_mapping, hsa_brite, on='keggId')

In [30]:
gene_mapping.head()

,geneId,geneName,keggId,keggBriteId
0,ENSG00000187961.13,KLHL17,hsa:339451,br:hsa00001
1,ENSG00000187961.13,KLHL17,hsa:339451,br:hsa04121
2,ENSG00000188290.10,HES4,hsa:57801,br:hsa00001
3,ENSG00000188290.10,HES4,hsa:57801,br:hsa03000
4,ENSG00000187608.8,ISG15,hsa:9636,br:hsa00001


In [31]:
# Sanity check
sum(gene_mapping.duplicated())

0

In [32]:
print("Number of Hugo genes contained in the expression datasets that are associated with KEGG BRITE" +
      " IDs:", len(set(gene_mapping['geneName'])))

Number of Hugo genes contained in the expression datasets that are associated with KEGG BRITE IDs: 7495


In [33]:
print("Number of ENSEMBLE genes contained in the expression datasets that are associated with KEGG BRITE" +
      " IDs:", len(set(gene_mapping['geneId'])))

Number of ENSEMBLE genes contained in the expression datasets that are associated with KEGG BRITE IDs: 7509


### Mapping: KEGG BRITE id to KEGG BRITE functional hierarchies

We pre-process the KEGG Brite IDs contained in `gene_mapping` dataset, in order to match the KEGG Brite IDs contained in `brite` dataset:

In [34]:
# Check all KEGG Brite IDs contained in gene_mapping dataframe have the same length
id_len = len(gene_mapping['keggBriteId'][0])
all(gene_mapping['keggBriteId'].apply(lambda x: len(x) == id_len))

True

In [35]:
# Check all KEGG Brite IDs contained in brite dataframe have the same length
brite_id_len = len(brite['keggBriteId'][0])
all(brite['keggBriteId'].apply(lambda x: len(x.split('_')[0]) == brite_id_len))

True

In [36]:
# Extract the last four digits of each KEGG Brite ID
gene_mapping['keggBriteId'] = gene_mapping['keggBriteId'].apply(lambda x: x[-4:])

In [37]:
hsa_brite['keggBriteId'].apply(lambda x: x[-4:]).drop_duplicates()

0        0001
13910    1000
17827    0199
17888    0535
17941    0536
18143    1001
18667    3000
19802    1002
20352    1003
20576    1004
20655    1006
20670    1007
20728    1009
21185    3110
21408    4121
22225    2044
22241    4054
22301    2042
22307    2048
22310    3011
22497    3012
22590    3041
22948    3021
23194    4131
24566    3100
24880    2000
25546    3051
25621    3029
25987    3009
26232    3036
27423    3310
27471    0537
27556    4812
28064    3019
28505    3032
28630    4052
28867    3400
29186    4090
29628    4040
29938    4515
30130    4147
31292    4091
31428    4031
31620    3016
31783    4030
32600    4050
Name: keggBriteId, dtype: object

Now, we perform the final mapping:

In [38]:
gene_mapping = pd.merge(gene_mapping, brite, on='keggBriteId')

In [39]:
gene_mapping.head()

,geneId,geneName,keggId,keggBriteId,Functional Annotation Group,Functional Annotation Subgroup,Functional Annotation
0,ENSG00000187961.13,KLHL17,hsa:339451,0001,Genes and Proteins,Orthologs and modules,KEGG Orthology (KO)
1,ENSG00000188290.10,HES4,hsa:57801,0001,Genes and Proteins,Orthologs and modules,KEGG Orthology (KO)
2,ENSG00000187608.8,ISG15,hsa:9636,0001,Genes and Proteins,Orthologs and modules,KEGG Orthology (KO)
3,ENSG00000188157.13,AGRN,hsa:375790,0001,Genes and Proteins,Orthologs and modules,KEGG Orthology (KO)
4,ENSG00000186891.13,TNFRSF18,hsa:8784,0001,Genes and Proteins,Orthologs and modules,KEGG Orthology (KO)


In [40]:
# Sanity check
sum(gene_mapping.duplicated())

0

In [41]:
print("Number of Hugo genes contained in the expression datasets that are associated with KEGG BRITE" +
      " functional annotations:", len(set(gene_mapping['geneName'])))

Number of Hugo genes contained in the expression datasets that are associated with KEGG BRITE functional annotations: 7495


In [42]:
print("Number of ENSEMBLE genes contained in the expression datasets that are associated with KEGG BRITE" +
      " functional annotations:", len(set(gene_mapping['geneId'])))

Number of ENSEMBLE genes contained in the expression datasets that are associated with KEGG BRITE functional annotations: 7509


In [43]:
gene_mapping['Functional Annotation Group'].value_counts()

Genes and Proteins    18494
Name: Functional Annotation Group, dtype: int64

In [44]:
gene_mapping['Functional Annotation Subgroup'].value_counts()

Orthologs and modules                                 7938
Protein families: signaling and cellular processes    3715
Protein families: metabolism                          3619
Protein families: genetic information processing      3209
RNA family                                              13
Name: Functional Annotation Subgroup, dtype: int64

In [47]:
gene_mapping['Functional Annotation'].value_counts()

KEGG Orthology (KO)                                     7938
Enzymes                                                 2302
Membrane trafficking                                     770
Transcription factors                                    754
Exosome                                                  659
Chromosome and associated proteins                       611
Transporters                                             486
CD molecules                                             411
Cytoskeleton proteins                                    379
Peptidases                                               369
Protein kinases                                          365
G protein-coupled receptors                              343
Ubiquitin system                                         324
Protein phosphatases and associated proteins             275
Ion channels                                             265
Cell adhesion molecules                                  205
Cytokines and growth fac

## KEGG BRITE functional hierarchies dataset

Lastly, we create a final table that links the hierarchical functional information from KEGG BRITE to the genes present in the PanCancer gene-expression dataset.

In [45]:
%%time
exp_to_tree_map = pd.concat([non_brca_ex, brca_ex]).T
exp_to_tree_map['geneId'] = exp_to_tree_map.index
exp_to_tree_map['tamPixel'] = np.ones(exp_to_tree_map.shape[0])
exp_to_tree_map['order'] = pd.concat([non_brca_ex, brca_ex]).mean(axis=0) # sort genes by mean expression values 
exp_to_tree_map = pd.merge(gene_mapping, exp_to_tree_map, on='geneId')

CPU times: user 5.51 s, sys: 2.82 s, total: 8.34 s
Wall time: 4.23 s


In [46]:
exp_to_tree_map.head()

,geneId,geneName,keggId,keggBriteId,Functional Annotation Group,Functional Annotation Subgroup,Functional Annotation,TCGA-02-0047-01_NON_BRCA,TCGA-02-0055-01_NON_BRCA,TCGA-02-2483-01_NON_BRCA,...,TCGA-V7-A7HQ-01_BRCA,TCGA-W8-A86G-01_BRCA,TCGA-WT-AB41-01_BRCA,TCGA-WT-AB44-01_BRCA,TCGA-XX-A899-01_BRCA,TCGA-XX-A89A-01_BRCA,TCGA-Z7-A8R5-01_BRCA,TCGA-Z7-A8R6-01_BRCA,tamPixel,order
0,ENSG00000187961.13,KLHL17,hsa:339451,0001,Genes and Proteins,Orthologs and modules,KEGG Orthology (KO),1.3225,2.3135,2.5707,...,1.8404,3.9552,1.0847,1.8762,3.1860,2.6738,2.0360,3.2766,1.0,1.992469
1,ENSG00000187961.13,KLHL17,hsa:339451,4121,Genes and Proteins,Protein families: genetic information processing,Ubiquitin system,1.3225,2.3135,2.5707,...,1.8404,3.9552,1.0847,1.8762,3.1860,2.6738,2.0360,3.2766,1.0,1.992469
2,ENSG00000188290.10,HES4,hsa:57801,0001,Genes and Proteins,Orthologs and modules,KEGG Orthology (KO),4.1604,3.6148,3.8729,...,5.5085,4.7296,2.9581,3.9910,4.4758,2.8681,4.4108,5.1538,1.0,2.791702
3,ENSG00000188290.10,HES4,hsa:57801,3000,Genes and Proteins,Protein families: genetic information processing,Transcription factors,4.1604,3.6148,3.8729,...,5.5085,4.7296,2.9581,3.9910,4.4758,2.8681,4.4108,5.1538,1.0,2.791702
4,ENSG00000187608.8,ISG15,hsa:9636,0001,Genes and Proteins,Orthologs and modules,KEGG Orthology (KO),5.8166,6.9599,5.9072,...,7.6990,5.2540,8.9442,5.5448,7.2307,5.7552,5.4380,7.1096,1.0,6.491173


In [47]:
%%time
# Sanity check
sum(exp_to_tree_map.duplicated())

CPU times: user 4.57 s, sys: 654 ms, total: 5.22 s
Wall time: 5.22 s


0

In [48]:
# Sanity check (number of ENSEMBLE genes contained in the final dataset)
len(set(exp_to_tree_map['geneId']))

7509

In [49]:
exp_to_tree_map.shape

(18494, 10544)

Finally, we save the final dataset, which is used as input data in `2-KEGG_BRITE_Treemap` notebook. This R notebook implements the generation of the final gene-expression treemap images.

In [50]:
%%time
# Save dataset
exp_to_tree_map.to_csv("./KEGG_exp_to_tree_map.csv", index=False)

CPU times: user 2min 8s, sys: 544 ms, total: 2min 9s
Wall time: 2min 11s
